In [2]:
import os

In [3]:
%pwd

'c:\\Users\\singh\\OneDrive\\Desktop\\Text-Summarize\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\singh\\OneDrive\\Desktop\\Text-Summarize'

In [39]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [40]:
os.chdir("src")

In [41]:
from TextSummarization.constants import *


In [42]:
from TextSummarization.utils.common import read_yaml, create_directories

In [43]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_NAME,
        params_filepath = PARAMS_FILE_NAME):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [11]:
!pip install transformers datasets

In [44]:
import os
from TextSummarization.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [45]:

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
"""
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self, example_batch):
        # Tokenize dialogues (inputs) and summaries (labels)
        inputs = self.tokenizer(
            example_batch["dialogue"],
            max_length=1024,
            truncation=True,
            padding="max_length"
        )

        targets = self.tokenizer(
            example_batch["summary"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        inputs["labels"] = targets["input_ids"]
        return inputs

    def convert(self):
        dataset_samsum = load_dataset(
            "csv",
            data_files={
                "train": "artifacts/data_ingestion/train.csv",
                "validation": "artifacts/data_ingestion/validation.csv",
                "test": "artifacts/data_ingestion/test.csv"
            }
        )

    

        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features,
            batched=True
        )

        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
"""


'\nclass DataTransformation:\n    def __init__(self, config: DataTransformationConfig):\n        self.config = config\n        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)\n\n\n\n    def convert_examples_to_features(self, example_batch):\n        # Tokenize dialogues (inputs) and summaries (labels)\n        inputs = self.tokenizer(\n            example_batch["dialogue"],\n            max_length=1024,\n            truncation=True,\n            padding="max_length"\n        )\n\n        targets = self.tokenizer(\n            example_batch["summary"],\n            max_length=128,\n            truncation=True,\n            padding="max_length"\n        )\n\n        inputs["labels"] = targets["input_ids"]\n        return inputs\n\n    def convert(self):\n        dataset_samsum = load_dataset(\n            "csv",\n            data_files={\n                "train": "artifacts/data_ingestion/train.csv",\n                "validation": "artifacts/data_ingestion/validatio

In [46]:
"""
import os
from datasets import load_dataset
from transformers import AutoTokenizer

class DataTransformation:
    def __init__(self, config):
        self.config = config
        # Load tokenizer once (replace with your model name if needed)
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        # Ensure all values are strings (avoids TypeError: TextEncodeInput...)
        dialogues = [str(x) if x is not None else "" for x in example_batch["dialogue"]]
        summaries = [str(x) if x is not None else "" for x in example_batch["summary"]]

        # Tokenize dialogues (inputs)
        inputs = self.tokenizer(
            dialogues,
            max_length=1024,
            truncation=True,
            padding="max_length"
        )

        # Tokenize summaries (labels)
        targets = self.tokenizer(
            summaries,
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        # Add labels to inputs
        inputs["labels"] = targets["input_ids"]
        return inputs

    def convert(self):
        # Load dataset from CSVs
        dataset_samsum = load_dataset(
            "csv",
            data_files={
                "train": "artifacts/data_ingestion/train.csv",
                "validation": "artifacts/data_ingestion/validation.csv",
                "test": "artifacts/data_ingestion/test.csv"
            }
        )

        # Apply tokenization
        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features,
            batched=True,
            remove_columns=["dialogue", "summary"]  # keep only tokenized features
        )

        # Save processed dataset
        dataset_samsum_pt.save_to_disk(
            os.path.join(self.config.root_dir, "samsum_dataset")
        )
"""


'\nimport os\nfrom datasets import load_dataset\nfrom transformers import AutoTokenizer\n\nclass DataTransformation:\n    def __init__(self, config):\n        self.config = config\n        # Load tokenizer once (replace with your model name if needed)\n        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)\n\n    def convert_examples_to_features(self, example_batch):\n        # Ensure all values are strings (avoids TypeError: TextEncodeInput...)\n        dialogues = [str(x) if x is not None else "" for x in example_batch["dialogue"]]\n        summaries = [str(x) if x is not None else "" for x in example_batch["summary"]]\n\n        # Tokenize dialogues (inputs)\n        inputs = self.tokenizer(\n            dialogues,\n            max_length=1024,\n            truncation=True,\n            padding="max_length"\n        )\n\n        # Tokenize summaries (labels)\n        targets = self.tokenizer(\n            summaries,\n            max_length=128,\n         

In [48]:
os.chdir("../")

In [26]:
!pip install tiktoken protobuf
!pip install --upgrade transformers tokenizers


In [27]:
!pip install sentencepiece


In [47]:
%pwd

'c:\\Users\\singh\\OneDrive\\Desktop\\Text-Summarize\\src'

In [49]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-08-19 19:28:23,758 : INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-08-19 19:28:23,779 : INFO:common:yaml file: params.yaml loaded successfully]
[2025-08-19 19:28:23,785 : INFO:common:created directory at: artifacts]
[2025-08-19 19:28:23,790 : INFO:common:created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\singh\OneDrive\Desktop\Text-Summarize\Text-Summarize\Lib\site-packages\transformers\tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 103322.91 examples/s]
